# Process US County-level Data from NYT

In [39]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import mapclassify as mc

Step0:  
Read data and get basic county groups

0.0 Report Data

In [40]:
# Read updated CSV file
df_counties_original = pd.read_csv("./us-counties.csv", dtype={"date": np.string_, 
                                                             "county": np.string_,
                                                             "state": np.string_,
                                                             "fips": np.string_,
                                                            "cases": np.int32,
                                                            "deaths": np.int32})
# Change 'Unknown' fips to '00000' 
df_counties_original['fips'] = df_counties_original['fips'].replace({np.nan: '00000'})
df_counties_original.sort_values("fips", ascending=True)

,date,county,state,fips,cases,deaths
96752,2020-04-28,Kansas City,Missouri,00000,511,16
239806,2020-06-14,Unknown,Utah,00000,24,5
239346,2020-06-14,Unknown,Rhode Island,00000,1870,36
239334,2020-06-14,Unknown,Puerto Rico,00000,234,0
238999,2020-06-14,Unknown,Northern Mariana Islands,00000,30,2
...,...,...,...,...,...,...
252282,2020-06-18,St. Thomas,Virgin Islands,78030,39,0
224297,2020-06-09,St. Thomas,Virgin Islands,78030,39,0
40350,2020-04-07,St. Thomas,Virgin Islands,78030,39,0
187332,2020-05-28,St. Thomas,Virgin Islands,78030,38,0


0.1 GeoJSON Data

In [41]:
# Read GeoJson data
old_counties_geojson_df = gpd.read_file(r"./counties_update_new.geojson")
old_counties_geojson_df.head(5)

,NAME,state_name,GEOID,population,geometry
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ..."
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ..."
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ..."
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ..."
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ..."


0.2 Group Data to Counties

In [42]:
# Transform cases/deaths data table to pivot table using fips as main index
pivot_counties = pd.pivot_table(df_counties_original , index=['state','county','fips'],
                       columns=['date'])
pivot_counties = pivot_counties.replace(np.nan, 0)
pivot_counties

cases                                              \
date                   2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                                                                               \
date                   2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county   fips                                                           
Alabama Autauga  01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin  01003        0.0        0.0        0.0        0.0        0.0   
        Barbour  01005        0.0        0.0        0.0        0.0        0.0   
        Bibb     01007        0.0        0.0        0.0        0.0        0.0   
        Blount   01009        0.0        0.0        0.0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
Wyoming Teton    56039        0.0        0.0        0.0        0.0        0.0   
        Uinta    56041        0.0        0.0        0.0        0.0        0.0   
        Unknown  00000        0.0        0.0        0.0        0.0        0.0   
        Washakie 56043        0.0        0.0        0.0        0.0        0.0   
        Weston   56045        0.0        0.0        0.0        0.0        0.0   

                        ...     deaths                                   \
date                    ... 2020-08-04 2020-08-05 2020-08-06 2020-08-07   
state   county   fips   ...                                               
Alabama Autauga  01001  ...       21.0       22.0       22.0       22.0   
        Baldwin  01003  ...       24.0       24.0       25.0       25.0   
        Barbour  01005  ...        5.0        5.0        5.0        5.0   
        Bibb     01007  ...        3.0        4.0        4.0        4.0   
        Blount   01009  ...        3.0        3.0        3.0        3.0   
...                     ...        ...        ...        ...        ...   
Wyoming Teton    56039  ...        1.0        1.0        1.0        1.0   
        Uinta    56041  ...        1.0        1.0        1.0        1.0   
        Unknown  00000  ...        0.0        0.0        0.0        0.0   
        Washakie 56043  ...        5.0        5.0        5.0        5.0   
        Weston   56045  ...        0.0        0.0        0.0        0.0   

                                                                               \
date                   2020-08-08 2020-08-09 2020-08-10 2020-08-11 2020-08-12   
state   county   fips                                                           
Alabama Autauga  01001       22.0       22.0       22.0       23.0       23.0   
        Baldwin  01003       26.0       27.0       28.0       32.0       32.0   
        Barbour  01005        5.0        5.0        5.0        6.0        6.0   
        Bibb     01007        5.0        5.0        5.0        6.0        6.0   
        Blount   01009        4.0        4.0        4.0        5.0        5.0   
...                           ...        ...        ...    

0.3 Get Date Info

In [43]:
# Get dates from data table
date= pivot_counties['cases'].columns
date

Index(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
       '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30',
       ...
       '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07', '2020-08-08',
       '2020-08-09', '2020-08-10', '2020-08-11', '2020-08-12', '2020-08-13'],
      dtype='object', name='date', length=206)

In [44]:
# Sort dates and get the start/end
date_str = np.sort(date)
dt_str_start=np.min(date_str)
dt_str_end=np.max(date_str)

In [45]:
# Format dates and create a list
start = datetime.strptime(dt_str_start, "%Y-%m-%d")
end = datetime.strptime(dt_str_end, "%Y-%m-%d")
dt_range = pd.date_range(start=start,end=end)
print(len(dt_range), dt_range)
dt_range_str = list(map(lambda x: x.strftime("%Y-%m-%d"), dt_range.tolist()))
print(len(dt_range_str), dt_range_str)

206 DatetimeIndex(['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
               '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
               '2020-01-29', '2020-01-30',
               ...
               '2020-08-04', '2020-08-05', '2020-08-06', '2020-08-07',
               '2020-08-08', '2020-08-09', '2020-08-10', '2020-08-11',
               '2020-08-12', '2020-08-13'],
              dtype='datetime64[ns]', length=206, freq='D')
206 ['2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-

Step1:  
Add Cases Time Series, First Case Date, Death Time Series, First Death Date

In [46]:
# import json
# pivot_counties['cases_ts'] = json.dumps({"values": pivot_counties['cases'].values.tolist()[0]})
# pivot_counties['deaths_ts'] =  json.dumps({"values": pivot_counties['deaths'].values.tolist()[0]})

# Create time series of cases/deaths
pivot_counties['cases_ts'] =  pivot_counties['cases'].values.tolist()
pivot_counties['deaths_ts'] =  pivot_counties['deaths'].values.tolist()

In [47]:
# Get the date of first case/death
pivot_counties['dt_first_case'] = (pivot_counties['cases'] > 0).idxmax(axis=1)
pivot_counties['dt_first_death'] = (pivot_counties['deaths'] > 0).idxmax(axis=1)
# For death reports, deal with no deaths counties
pivot_counties.loc[pivot_counties['deaths'].iloc[:, -1] <= 0, 'dt_first_death'] = np.nan
pivot_counties.head(5)

cases                                              \
date                  2020-01-21 2020-01-22 2020-01-23 2020-01-24 2020-01-25   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                                                                              \
date                  2020-01-26 2020-01-27 2020-01-28 2020-01-29 2020-01-30   
state   county  fips                                                           
Alabama Autauga 01001        0.0        0.0        0.0        0.0        0.0   
        Baldwin 01003        0.0        0.0        0.0        0.0        0.0   
        Barbour 01005        0.0        0.0        0.0        0.0        0.0   
        Bibb    01007        0.0        0.0        0.0        0.0        0.0   
        Blount  01009        0.0        0.0        0.0        0.0        0.0   

                       ...     deaths                                   \
date                   ... 2020-08-08 2020-08-09 2020-08-10 2020-08-11   
state   county  fips   ...                                               
Alabama Autauga 01001  ...       22.0       22.0       22.0       23.0   
        Baldwin 01003  ...       26.0       27.0       28.0       32.0   
        Barbour 01005  ...        5.0        5.0        5.0        6.0   
        Bibb    01007  ...        5.0        5.0        5.0        6.0   
        Blount  01009  ...        4.0        4.0        4.0        5.0   

                                             \
date                  2020-08-12 2020-08-13   
state   county  fips                          
Alabama Autauga 01001       23.0       23.0   
        Baldwin 01003       32.0       32.0   
        Barbour 01005        6.0        6.0   
        Bibb    01007        6.0        6.0   
        Blount  01009        5.0        5.0   

                                                                cases_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                                               deaths_ts  \
date                                                                       
state   county  fips                                                       
Alabama Autauga 01001  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Baldwin 01003  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Barbour 01005  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Bibb    01007  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
        Blount  01009  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                      dt_first_case dt_first_death  
date                                                
state   county  fips                                
Alabama Autauga 01001    2020-03-24     2020-04-06  
        Baldwin 01003    2020-03-14     2020-04-01  
        Barbour 01005    2020-04-03     2020-04-29  
        Bibb    01007    2020-03-30     2020-05-08  
        Blount  01009    2020-03-25     2020-05-17  

[5 rows x 416 columns]

Step2:  
Add today_case, today_new_case, today_death, today_new_death

In [48]:
# Get the latest case/death
yesterday = date.values[-2]
pivot_counties['today_case'] = pivot_counties['cases'][dt_str_end]
pivot_counties['today_new_case'] = pivot_counties['cases'][dt_str_end] - pivot_counties['cases'][yesterday]
pivot_counties['today_death'] = pivot_counties['deaths'][dt_str_end]
pivot_counties['today_new_death'] = pivot_counties['deaths'][dt_str_end] - pivot_counties['deaths'][yesterday]

Step3:  
Extract necessary columns

In [49]:
# Only keep the necessary columns
report_df = pivot_counties[['cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']]
report_df = report_df.reset_index()
report_df.columns = ['state','county','fips','cases_ts','deaths_ts','dt_first_case','dt_first_death','today_case','today_new_case', 'today_death','today_new_death']
report_df.head(5)

,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Alabama,Autauga,01001,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-24,2020-04-06,1250.0,9.0,23.0,0.0
1,Alabama,Baldwin,01003,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-14,2020-04-01,3813.0,37.0,32.0,0.0
2,Alabama,Barbour,01005,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-04-03,2020-04-29,651.0,5.0,6.0,0.0
3,Alabama,Bibb,01007,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-30,2020-05-08,469.0,5.0,6.0,0.0
4,Alabama,Blount,01009,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2020-03-25,2020-05-17,977.0,10.0,5.0,0.0


Step4:  
Assign geometries to counties

In [50]:
# Merge case dataframe and geometry
final_df = pd.merge(old_counties_geojson_df[["NAME", "state_name", "GEOID", "population", "geometry"]], report_df, how='left', left_on=['NAME','state_name'], right_on = ['county','state'])
final_df.columns

Index(['NAME', 'state_name', 'GEOID', 'population', 'geometry', 'state',
       'county', 'fips', 'cases_ts', 'deaths_ts', 'dt_first_case',
       'dt_first_death', 'today_case', 'today_new_case', 'today_death',
       'today_new_death'],
      dtype='object')

Step5:  
Deal with nan values

In [51]:
# Replace NAN values with 0
final_df[['today_case','today_new_case','today_death','today_new_death']] = final_df[['today_case','today_new_case','today_death','today_new_death']].replace(np.nan,0)
final_df['fips'] = final_df['fips'].replace(np.nan, final_df['GEOID'])

In [52]:
# Set up template
template =final_df[~final_df['cases_ts'].isna()]['cases_ts']
template = template.iloc[0]

In [53]:
for x in range(0,len(template)):
    template[x] = 0
template = ','.join(map(str, template))

In [54]:
# Change list format to string with commas
final_df['cases_ts'] = final_df['cases_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df['deaths_ts'] = final_df['deaths_ts'].apply(lambda x: ','.join(map(str, x)) if type(x) is list else template)
final_df.head(5)

,NAME,state_name,GEOID,population,geometry,state,county,fips,cases_ts,deaths_ts,dt_first_case,dt_first_death,today_case,today_new_case,today_death,today_new_death
0,Bladen,North Carolina,37017,33778,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",North Carolina,Bladen,37017,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-03,2020-05-06,634.0,7.0,6.0,1.0
1,Stanly,North Carolina,37167,61114,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",North Carolina,Stanly,37167,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-04-09,1171.0,68.0,31.0,0.0
2,Summit,Ohio,39153,541810,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",Ohio,Summit,39153,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-14,2020-03-27,3712.0,45.0,224.0,0.0
3,Sullivan,Pennsylvania,42113,6177,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",Pennsylvania,Sullivan,42113,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-04-04,NaN,10.0,0.0,0.0,0.0
4,Upshur,Texas,48459,40769,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",Texas,Upshur,48459,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",2020-03-20,2020-07-25,262.0,5.0,2.0,0.0


Step6:  
Finalize the dataframe

In [55]:
# Keep and add essential columns
final_df=final_df[["NAME", "state_name", "population", "fips", "dt_first_case", "dt_first_death", "cases_ts", "deaths_ts", 
                   'today_case','today_new_case','today_death','today_new_death', "geometry"]]
final_df['start'] = dt_str_start
final_df['end'] = dt_str_end
final_df['dt_unit'] = "day"
final_df.head(1)

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",634.0,7.0,6.0,1.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-08-13,day


In [56]:
# Calculate weekly change rates of cases and form a time-series list
final_df['change_ts'] = ""
for x in range(0, len(final_df)):
    newCaseList = [0]
    changeRateList = []
    caseStrList = final_df.loc[x,'cases_ts'].split(",")
    for i in range(1, len(caseStrList)):
        dailyNewCase = float(caseStrList[i]) - float(caseStrList[i-1])
        newCaseList.append(dailyNewCase)
    for i in range(0, len(newCaseList)):
        if i < 13:
            changeRate = 0
        else:
            currentWeekSum = 0
            previousWeekSum = 0
            for j in range(0,7):
                currentWeekSum = currentWeekSum + newCaseList[i-j]
            for k in range(7,14):
                previousWeekSum = previousWeekSum + newCaseList[i-k]
            if previousWeekSum == 0 and currentWeekSum == 0:
                changeRate = 0
            elif previousWeekSum == 0 and currentWeekSum != 0:
                changeRate = currentWeekSum
            else:
                changeRate = round(currentWeekSum/previousWeekSum-1,2)
        changeRateList.append(changeRate)
        changeRateStrList = list(map(str, changeRateList))
#     print(','.join(changeRateStrList))
    final_df.loc[x,'change_ts'] = ','.join(changeRateStrList)

final_df

,NAME,state_name,population,fips,dt_first_case,dt_first_death,cases_ts,deaths_ts,today_case,today_new_case,today_death,today_new_death,geometry,start,end,dt_unit,change_ts
0,Bladen,North Carolina,33778,37017,2020-04-03,2020-05-06,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",634.0,7.0,6.0,1.0,"MULTIPOLYGON (((-78.90200 34.83527, -78.79960 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,Stanly,North Carolina,61114,37167,2020-03-20,2020-04-09,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1171.0,68.0,31.0,0.0,"MULTIPOLYGON (((-80.49738 35.20210, -80.29542 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,Summit,Ohio,541810,39153,2020-03-14,2020-03-27,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",3712.0,45.0,224.0,0.0,"MULTIPOLYGON (((-81.68699 41.13596, -81.68495 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3,Sullivan,Pennsylvania,6177,42113,2020-04-04,NaN,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",10.0,0.0,0.0,0.0,"MULTIPOLYGON (((-76.81373 41.59003, -76.22014 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
4,Upshur,Texas,40769,48459,2020-03-20,2020-07-25,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",262.0,5.0,2.0,0.0,"MULTIPOLYGON (((-95.15274 32.66095, -95.15211 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3212,Seward,Nebraska,17127,31159,2020-04-05,2020-04-21,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",127.0,2.0,1.0,0.0,"MULTIPOLYGON (((-97.36812 41.04695, -96.91094 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3213,Burke,North Carolina,89712,37023,2020-03-25,2020-04-05,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",1679.0,3.0,27.0,1.0,"MULTIPOLYGON (((-81.90665 35.88338, -81.94319 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3214,Sumter,Georgia,30352,13261,2020-03-20,2020-03-28,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",768.0,-16.0,56.0,0.0,"MULTIPOLYGON (((-84.43301 32.04196, -84.43121 ...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
3215,Kansas City,Missouri,491918,00000,2020-03-20,2020-04-02,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",7285.0,208.0,82.0,2.0,"POLYGON ((-94.44849 38.93168, -94.44869 38.928...",2020-01-21,2020-08-13,day,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."


Step 7:  
Output file

In [57]:
# Save file
final_df.to_file(r"./nyt_counties_data.geojson", driver='GeoJSON', encoding='utf-8')
print("done")

done
